In [1]:
import pandas as pd
import numpy as np
import re
from konlpy.tag import Okt
from gensim.models import Word2Vec

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('./src/네이버종토방댓글_삼성전자_10000.csv')
df = df[df['댓글'] != 'error'] # 전처리 에러 제거
df = df.dropna() # 결측치 제거
df['한글댓글'] = df['댓글'].str.replace('\[삭제된 게시물의 답글\]',' ') # 답글 태그 제거
df['한글댓글'] = df['한글댓글'].str.replace('답글:',' ')
df['한글댓글'] = df['한글댓글'].str.replace('[^가-힣]',' ').str.replace(' +',' ').str.strip() # 한글만 남기기
df = df[df['한글댓글'] != ''] # 결측치 제거
df = df.reset_index(drop=True) # 인덱스 리셋
df

,날짜,댓글,조회수,좋아요,싫어요,한글댓글
0,2022-05-02,개미들 바닥에서 잘샀네..,29,0,2,개미들 바닥에서 잘샀네
1,2022-05-02,* 내일 상한가 : 코스모화학...,17,0,0,내일 상한가 코스모화학
2,2022-05-02,문재앙이 가장 이해 안가는일 양산 아방궁...,29,4,3,문재앙이 가장 이해 안가는일 양산 아방궁
3,2022-05-02,"●[속보]문재인,이재용 사면 안한다●",50,0,0,속보 문재인 이재용 사면 안한다
4,2022-05-02,사면 나가리~,29,1,0,사면 나가리
...,...,...,...,...,...,...
198005,2021-12-14,양성전자가자!! 10만가자!!,325,0,0,양성전자가자 만가자
198006,2021-12-14,답글:[삭제된 게시물의 답글]쳐맞고 있는건 석...,294,0,1,쳐맞고 있는건 석
198007,2021-12-14,규모 4.9 지진,345,2,3,규모 지진
198008,2021-12-14,지진났어도 뭉가리는호주에서 신나게ㅡ,332,3,3,지진났어도 뭉가리는호주에서 신나게


In [3]:
okt = Okt()

tag_list = ['Noun','Verb','Adjective','VerbPrefix']

tokenized_data = []
for i in range(df.shape[0]):
    tokenized_sentence = okt.pos(df['한글댓글'][i], stem=True) # 토큰화
    tag_checked_sentence = []
    for j in tokenized_sentence:
        x,y = j
        if y in tag_list:
            tag_checked_sentence.append(x)
    tokenized_data.append(tag_checked_sentence)
    if i % 10000 == 0:
        print(f'{i}개 토큰화 완료')

0개 토큰화 완료
10000개 토큰화 완료
20000개 토큰화 완료
30000개 토큰화 완료
40000개 토큰화 완료
50000개 토큰화 완료
60000개 토큰화 완료
70000개 토큰화 완료
80000개 토큰화 완료
90000개 토큰화 완료
100000개 토큰화 완료
110000개 토큰화 완료
120000개 토큰화 완료
130000개 토큰화 완료
140000개 토큰화 완료
150000개 토큰화 완료
160000개 토큰화 완료
170000개 토큰화 완료
180000개 토큰화 완료
190000개 토큰화 완료


In [4]:
df['토큰화댓글'] = tokenized_data
df = df[df['토큰화댓글'].str.len() > 1]
df

,날짜,댓글,조회수,좋아요,싫어요,한글댓글,토큰화댓글
0,2022-05-02,개미들 바닥에서 잘샀네..,29,0,2,개미들 바닥에서 잘샀네,"[개미, 바닥, 잘, 사다]"
1,2022-05-02,* 내일 상한가 : 코스모화학...,17,0,0,내일 상한가 코스모화학,"[내일, 상한, 코스모, 화학]"
2,2022-05-02,문재앙이 가장 이해 안가는일 양산 아방궁...,29,4,3,문재앙이 가장 이해 안가는일 양산 아방궁,"[재앙, 가장, 이해, 안, 가다, 일, 양산, 아방궁]"
3,2022-05-02,"●[속보]문재인,이재용 사면 안한다●",50,0,0,속보 문재인 이재용 사면 안한다,"[속보, 문재인, 이재용, 사면, 안, 하다]"
4,2022-05-02,사면 나가리~,29,1,0,사면 나가리,"[사면, 나가다]"
...,...,...,...,...,...,...,...
198005,2021-12-14,양성전자가자!! 10만가자!!,325,0,0,양성전자가자 만가자,"[자가, 만가]"
198006,2021-12-14,답글:[삭제된 게시물의 답글]쳐맞고 있는건 석...,294,0,1,쳐맞고 있는건 석,"[치다, 맞다, 있다, 석]"
198007,2021-12-14,규모 4.9 지진,345,2,3,규모 지진,"[규모, 지진]"
198008,2021-12-14,지진났어도 뭉가리는호주에서 신나게ㅡ,332,3,3,지진났어도 뭉가리는호주에서 신나게,"[지진, 나다, 뭉, 가리다, 호주, 신나다]"


In [5]:
df.to_csv("./src/네이버종토방댓글_삼성전자_10000_태그토큰화.csv", index=False)